# Selection of Calibration Windows for Day-Ahead Electricity Price Forecasting
## by Grzegorz Marcjasz, Tomasz Serafin and Rafał Weron 

In [14]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import median_abs_deviation

In [30]:
df_day_ahead_epex = pd.read_csv('Day_Ahead_Epex.csv', sep=';')
df_NP2018 = pd.read_csv('NP2018.csv', header=None)
df_PJM = pd.read_csv('PJM.csv', sep=';')

In [3]:
df_day_ahead_epex.head()

,Date,hour,Spot DE.AT_price,day
0,20150101,1,25.02,5
1,20150101,2,18.29,5
2,20150101,3,16.04,5
3,20150101,4,14.60,5
4,20150101,5,14.95,5


In [4]:
df_NP2018.head()
columns = ['Date', 'Hour', 'Price', "load", "placeholder2"]
df_NP2018.columns = columns
df_NP2018.head()

,Date,Hour,Price,load,placeholder2
0,20130101,1,31.05,42497.0,2798.0
1,20130101,2,30.47,41463.0,2417.0
2,20130101,3,28.92,40812.0,2036.0
3,20130101,4,27.88,40246.0,1706.0
4,20130101,5,26.96,40088.0,1427.0


In [5]:
df_PJM.tail()

,date,hour,comed price,rto load forecast,comed load foecast,day
63547,20180402,19,28.574056,92745.0,10959.0,2
63548,20180402,20,28.555229,93931.0,11448.0,2
63549,20180402,21,25.727719,91143.0,11432.0,2
63550,20180402,22,21.196351,86108.0,11040.0,2
63551,20180402,23,18.031368,80994.0,10403.0,2


In [31]:
df_PJM['sunday'] = np.where(df_PJM['day'] == 7, 1, 0)
df_PJM['saturday'] = np.where(df_PJM['day'] == 6, 1, 0)
df_PJM['monday'] = np.where(df_PJM['day'] ==  1, 1, 0)


In [32]:
price = df_PJM["comed price"]
exogenos = df_PJM["rto load forecast"]
sunday = df_PJM["sunday"]
saturday = df_PJM["saturday"]
monday = df_PJM["monday"]
df_PJM['Min_Comed_Price'] = df_PJM.groupby('date')['comed price'].transform('min')

In [33]:
df_PJM.head()

,date,hour,comed price,rto load forecast,comed load foecast,day,sunday,saturday,monday,Min_Comed_Price
0,20110102,0,17.141179,68590.0,11432.0,1,0,0,1,10.690926
1,20110102,1,14.179616,65896.0,10862.0,1,0,0,1,10.690926
2,20110102,2,12.736793,64511.0,10486.0,1,0,0,1,10.690926
3,20110102,3,11.096262,63935.0,10318.0,1,0,0,1,10.690926
4,20110102,4,10.690926,64321.0,10208.0,1,0,0,1,10.690926


### Arx1

#### df_PJM

In [34]:
# Extract 'comed price' column and convert to numpy array
prices = df_PJM['comed price'].values

def normalize_prices(prices):
    a = np.median(prices)
    b = median_abs_deviation(prices)
    p = 1/b * (prices - a)
    return p, a, b

def apply_vst(p): # apply the area hyperbolic sine transformation to the normalized prices
    X = np.arcsinh(p)
    return X

def inverse_transform(forecast, a, b): # apply the inverse of VST to the forecast to obtain the price prediction
    price_predictions = b * np.sinh(forecast) + a
    return price_predictions

# Normalize prices
p, a, b = normalize_prices(prices)

# Apply VST
X = apply_vst(p)

# Update DataFrame
df_PJM['comed price'] = X

In [35]:
df_PJM.head()

,date,hour,comed price,rto load forecast,comed load foecast,day,sunday,saturday,monday,Min_Comed_Price
0,20110102,0,-1.387638,68590.0,11432.0,1,0,0,1,10.690926
1,20110102,1,-1.602532,65896.0,10862.0,1,0,0,1,10.690926
2,20110102,2,-1.693663,64511.0,10486.0,1,0,0,1,10.690926
3,20110102,3,-1.788647,63935.0,10318.0,1,0,0,1,10.690926
4,20110102,4,-1.810860,64321.0,10208.0,1,0,0,1,10.690926


In [36]:
load = df_PJM["rto load forecast"]
errors = []
T = 150
for h in range(24):
    p_hour = df_PJM.loc[h::24,'comed price'].values # hourly data selection
    exogenos = df_PJM.loc[h::24,'rto load forecast'].values
    sunday = df_PJM.loc[h::24,'sunday'].values
    monday = df_PJM.loc[h::24,'monday'].values
    saturday = df_PJM.loc[h::24,'saturday'].values
    c_d_min = df_PJM.loc[h::24,'Min_Comed_Price'].values
    h_errors = []
    # Initialize an empty list to store the forecasts
    forecasts = []
    for day in range(T,len(price)//24):
            cal_data = price[(day-T):day].values # calibration data 
            Y = cal_data[7:T]
            X1 = cal_data[6:T-1] 
            X2 = cal_data[5:T-2] 
            X3 = cal_data[0:T-7]
            X4 = c_d_min[7:T]
            X5 = exogenos[7:T]  #C_d_h
            X6 = sunday[7:T] # sunday
            X7 = monday[7:T] # monday
            X8 = saturday[7:T] # saturday
            X0 = np.ones(np.size(X1)) 
            X = np.stack([X0,X1,X2, X3,X4,X5, X6, X7, X8],axis = 1)
            betas = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T,Y))
            X_fut = np.array([1,cal_data[T-1],cal_data[T-2],cal_data[T-7],c_d_min[day], exogenos[day],sunday[day],monday[day],saturday[day]])
            forecast = np.dot(X_fut,betas)
            forecasts.append(forecast)
            real = price[day]
            err = real - forecast
            errors1 = (np.abs(err))
            errors.append(errors1)
print("errors for model 1: " , np.mean(errors))



errors for model 1:  3.095383368363987


In [37]:
# Convert the list of forecasts to a numpy array
forecasts = np.array(forecasts)

# Apply inverse transformation
price_predictions = inverse_transform(forecasts, a, b)

### 2. Methodology

#### 2.1. Preliminaries

In [ ]:
def normalize_prices(prices):
    a = np.median(prices)
    b = median_abs_deviation(prices)
    p = 1/b * (prices - a)
    return p, a, b

def apply_vst(p): # apply the area hyperbolic sine transformation to the normalized prices
    X = np.arcsinh(p)
    return X

def inverse_transform(forecast, a, b): # apply the inverse of VST to the forecast to obtain the price prediction
    price_predictions = b * np.sinh(forecast) + a
    return price_predictions

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import median_abs_deviation



# Steps for model training and forecasting go here

# Apply inverse transformation
# price_predictions = inverse_transform(forecasts, a, b)

#### 2.2. Expert Models

##### ARX1

In [ ]:
# Prepare the data
data = prepare_data_ar1(data)

# Fit the AR1 model
results = fit_ar1(data)

# Print the model summary
print(results.summary())

##### ARX2

In [ ]:
# Prepare the data
data = prepare_data_ar2(data)

# Fit the AR2 model
results = fit_ar2(data)

# Print the model summary
print(results.summary())